In [1]:
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

### Passo 1: 
Abrir o arquivo e colocar os vertices e faces em uma lista

In [2]:
def read_obj(file):
    vertices = []
    faces = []

    with open(file, 'r') as f:

        for line in f:
            parts = line.split()

            if parts[0] == 'v':
                x, y, z = map(float, parts[1:4])
                vertices.append([x, y, z])
            
            elif parts[0] == 'f':
                v1, v2, v3 = map(int, parts[1:4])
                faces.append([v1-1, v2-1, v3-1])

    return vertices, faces


def write_obj(vertices, faces, name='refined_OBJ'):

    with open(name+'.txt', 'w') as f:
        for v in vertices:
            f.write(f'v {v[0]} {v[1]} {v[2]}\n')

        for face in faces:
            f.write(f'f {face[0]+1} {face[1]+1} {face[2]+1}\n')

### Passo 2:
Normalização dos pontos

In [3]:
def normalize(v):
    x, y, z = v

    d = math.sqrt(x**2 + y**2 + z**2)
    return [x/d, y/d, z/d]

### Passo 3:
Ponto médio entre os vértices

In [4]:
def middle(i, j, vertices, vin):

    '''
    Função para encontrar o ponto médio entre dois pontos
    Parâmetros:
    i, j: pontos que queremos encontrar o ponto médio
    vertices: lista de vertices
    vin: hash com os vértices que já foram visitados

    Retorno:
    Indice do vértice que está no ponto médio
    '''

    key = (min(i, j), max(i, j))

    #Se o vértice já foi calculado como ponto médio
    if key in vin:
        return vin[key]
    
    v1 = vertices[i]
    v2 = vertices[j]

    meio = [(v1[0] + v2[0]) / 2, (v1[1] + v2[1]) / 2, (v1[2] + v2[2]) / 2]
    meio = normalize(meio)

    vertices.append(meio)
    vin[key] = len(vertices) - 1

    return len(vertices) - 1

### Passo 4
Fazer o refinamento

In [5]:
def refinement(vertices, faces):
    new_faces = []
    vin = {}

    for a, b, c in faces:

        #Ponto médio de cada aresta
        ab = middle(a, b, vertices, vin)
        bc = middle(b, c, vertices, vin)
        ca = middle(c, a, vertices, vin)

        #Adicionar os 4 triângulos
        new_faces.append([a, ab, ca])
        new_faces.append([ab, b, bc])
        new_faces.append([ab, ca, bc])
        new_faces.append([ca, bc, c])

    return vertices, new_faces

### Passo 5
Plotar o refinamento

In [6]:
def plot_refinement(vertices=[], faces=[], name=None):
    if name:
        vertices, faces = read_obj(name)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")

    mesh = [[vertices[i] for i in face] for face in faces]
    ax.add_collection3d(
        Poly3DCollection(mesh, alpha=0.6, edgecolor="k")
    )

    ax.set_box_aspect([1, 1, 1])
    plt.show()

### Passo 6
Função principal para executar tudo

In [7]:
def sphere(obj, new_obj='new_obj', level=1, plot=False):

    vertices, faces = read_obj(obj)

    vertices = [normalize(v) for v in vertices]

    for _ in range(level):
        vertices, faces = refinement(vertices, faces)

    write_obj(vertices, faces, new_obj)

    if plot:
        plot_refinement(vertices, faces)

In [8]:
sphere('OBJ.txt',plot=True, level=3)

FileNotFoundError: [Errno 2] No such file or directory: 'OBJ.txt'